可以整合的資料
- Finmind_daily 日交易：日 ok
- InsitutionalInvestorsBuySell 法人買賣：日 ok
- PurchaseShortSale 個股融資融卷：日 ok
- TaiwanStockDayTrading 當日沖銷交易標的及成交量值：日 ok
- TaiwanStockPER 個股股利、PER、PBR資料表：日 ok
- TaiwanStockShareHolding 股東結構表：日 ok
<br>

In [ ]:
def Merge(stock_id):
    import pandas as pd
    import datetime

    #stock_list
    stock_list_df = pd.read_csv("../Stock-Data/stock_list.csv", sep=",")
    mask = stock_list_df['stock_id']==int(stock_id)
    stock_list_df_mask=stock_list_df[mask]

    #Finmind_daily 日交易：日
    Finmind_daily_df = pd.read_csv("../Stock-Data/Finmind_daily/{}_daily.csv".format(stock_id), sep=",")
    Finmind_daily_df=Finmind_daily_df[['stock_id','date', 'Volume', 'Volume_Cash', 'Open','High', 'Low', 'Close', 'Change', 'Order']]
    Finmind_daily_df['date']=pd.to_datetime(Finmind_daily_df['date'])
    merge= pd.merge(stock_list_df_mask,Finmind_daily_df, on="stock_id",how='left')

    #TaiwanStockPER 個股股利、PER、PBR資料表：日
    TaiwanStockPER_df = pd.read_csv("../Stock-Data/TaiwanStockPER/{}_TaiwanStockPER.csv".format(stock_id), sep=",")
    TaiwanStockPER_df=TaiwanStockPER_df.drop(columns=['stock_id'])
    TaiwanStockPER_df['date']=pd.to_datetime(TaiwanStockPER_df['date'])
    TaiwanStockPER_df[TaiwanStockPER_df['date']>=datetime.datetime(2010,1,1)]
    
    merge2= pd.merge(merge, TaiwanStockPER_df, on="date",how='left')

    #InsitutionalInvestorsBuySell 法人買賣：日
    #轉置成橫向閱讀，且有的資料表格式不一
    List=['date','Dealer_buy','Dealer_Hedging_buy','Dealer_self_buy','Foreign_Dealer_Self_buy','Foreign_Investor_buy','Investment_Trust_buy','Dealer_sell','Dealer_Hedging_sell','Dealer_self_sell','Foreign_Dealer_Self_sell','Foreign_Investor_sell','Investment_Trust_sell']
    dataframe=pd.DataFrame(columns=List)

    #dataframe
    InstitutionalInvestorsBuySell_df = pd.read_csv("../Stock-Data/InstitutionalInvestorsBuySell/{}_InstitutionalInvestorsBuySell.csv".format(stock_id), sep=",")
    InstitutionalInvestorsBuySell_df_pivottable=InstitutionalInvestorsBuySell_df.pivot_table(index=['date'],columns=['name'],values=['sell','buy'],margins=False)
    df= pd.DataFrame(InstitutionalInvestorsBuySell_df_pivottable)
    df=df.reset_index(level=[0])

    List1=['date',]
    for i in range(1,len(df.columns)):
        List1.append(df.columns[i][1]+"_"+df.columns[i][0])
    df.columns=List1

    #df
    dataframe_df= pd.merge(df,dataframe,how='outer')
    #dataframe_df
    
    dataframe_df['date']=pd.to_datetime(dataframe_df['date'])
    dataframe_df[dataframe_df['date']>=datetime.datetime(2010,1,1)]
    
    merge3= pd.merge(merge2, dataframe_df, on="date",how='left')
        
    #TaiwanStockShareHolding 股東結構表：日
    TaiwanStockShareholding_df = pd.read_csv("../Stock-Data/TaiwanStockShareholding/{}_TaiwanStockShareholding.csv".format(stock_id), sep=",")
    TaiwanStockShareholding_df=TaiwanStockShareholding_df.drop(columns=['stock_id','stock_name','InternationalCode'])
    TaiwanStockShareholding_df.columns = ['date','ForeignInvestmentRemainingShares', 'ForeignInvestmentShares','ForeignInvestmentRemainRatio', 'ForeignInvestmentSharesRatio','ForeignInvestmentUpperLimitRatio', 'ChineseInvestmentUpperLimitRatio','NumberOfSharesIssued', 'RecentlyDeclareDate','StockShareholding_note']
    TaiwanStockShareholding_df['date']=pd.to_datetime(TaiwanStockShareholding_df['date'])
    TaiwanStockShareholding_df[TaiwanStockShareholding_df['date']>=datetime.datetime(2010,1,1)]
    
    merge4= pd.merge(merge3, TaiwanStockShareholding_df, on="date",how='left')
    
    try:
        #PurchaseShortSale 個股融資融卷：日，有的股票沒有
        PurchaseShortSale_df = pd.read_csv("../Stock-Data/PurchaseShortSale/{}_PurchaseShortSale.csv".format(stock_id), sep=",")
        PurchaseShortSale_df['date']=pd.to_datetime(PurchaseShortSale_df['date'])
        PurchaseShortSale_df=PurchaseShortSale_df.drop(columns=['stock_id', 'stock_name'])
        PurchaseShortSale_df[PurchaseShortSale_df['date']>=datetime.datetime(2010,1,1)]

        merge5= pd.merge(merge4, PurchaseShortSale_df, on="date",how='left')
        
    except FileNotFoundError:
        
        List=['date', 'MarginPurchaseBuy', 'MarginPurchaseCashRepayment','MarginPurchaseLimit', 'MarginPurchaseSell','MarginPurchaseTodayBalance', 'MarginPurchaseYesterdayBalance','OffsetLoanAndShort', 'ShortSaleBuy', 'ShortSaleCashRepayment','ShortSaleLimit', 'ShortSaleSell', 'ShortSaleTodayBalance','ShortSaleYesterdayBalance', 'Note']
        dataframe=pd.DataFrame(columns=List)
        
        merge5= pd.merge(merge4,dataframe,how='outer')
    
    try:
        #TaiwanStockDayTrading 當日沖銷交易標的及成交量值：日，有的股票沒有
        TaiwanStockDayTrading_df = pd.read_csv("../Stock-Data/TaiwanStockDayTrading/{}_TaiwanStockDayTrading.csv".format(stock_id), sep=",")
        TaiwanStockDayTrading_df['date']=pd.to_datetime(TaiwanStockDayTrading_df['date'])
        TaiwanStockDayTrading_df[TaiwanStockDayTrading_df['date']>=datetime.datetime(2010,1,1)]
        TaiwanStockDayTrading_df=TaiwanStockDayTrading_df.drop(columns=['stock_id'])
        TaiwanStockDayTrading_df.columns=['date','BuyAfterSale','Trading_Volume','BuyAmount','SellAmount']

        merge6= pd.merge(merge5, TaiwanStockDayTrading_df, on="date",how='left')
        merge.to_csv("merge_{}.csv".format(stock_id),encoding='utf-8')
        
    except FileNotFoundError:
        
        List=['date', 'BuyAfterSale', 'Trading_Volume', 'BuyAmount', 'SellAmount']
        dataframe=pd.DataFrame(columns=List)
        
        merge6= pd.merge(merge5,dataframe,how='outer')
        merge6_df.to_csv("merge_{}.csv".format(stock_id),encoding='utf-8')
    

In [ ]:
import csv
with open('../Stock-Data/stock_list.csv',encoding="utf-8") as f:
    list_of_stock = csv.DictReader(f)
    for obj in list_of_stock:
        Merge(obj['stock_id'])

# 因單一檔案不足 128MB，故先把所資料合併，視情況預備使用

In [ ]:
#用 linux 合併成一張 dataframe
#awk '(NR == 1) || (FNR > 1)' *.csv > merge.csv

In [11]:
import pandas as pd
stock_list_df = pd.read_csv("merge.csv", sep=",")
stock_list_df=stock_list_df.drop(columns='Unnamed: 0')
stock_list_df.to_csv("merge_dataframe.csv",encoding='utf-8')

# stock_list_df.columns
# Index(['stock_id', 'stock_name', 'ISIN_Code', 'Listing_date',
#        'Listing_category', 'Industry_category', 'CFICode', 'date', 'Volume',
#        'Volume_Cash', 'Open', 'High', 'Low', 'Close', 'Change', 'Order',
#        'dividend_yield', 'PER', 'PBR', 'Dealer_buy', 'Dealer_Hedging_buy',
#        'Dealer_self_buy', 'Foreign_Dealer_Self_buy', 'Foreign_Investor_buy',
#        'Investment_Trust_buy', 'Dealer_sell', 'Dealer_Hedging_sell',
#        'Dealer_self_sell', 'Foreign_Dealer_Self_sell', 'Foreign_Investor_sell',
#        'Investment_Trust_sell', 'ForeignInvestmentRemainingShares',
#        'ForeignInvestmentShares', 'ForeignInvestmentRemainRatio',
#        'ForeignInvestmentSharesRatio', 'ForeignInvestmentUpperLimitRatio',
#        'ChineseInvestmentUpperLimitRatio', 'NumberOfSharesIssued',
#        'RecentlyDeclareDate', 'StockShareholding_note', 'MarginPurchaseBuy',
#        'MarginPurchaseCashRepayment', 'MarginPurchaseLimit',
#        'MarginPurchaseSell', 'MarginPurchaseTodayBalance',
#        'MarginPurchaseYesterdayBalance', 'OffsetLoanAndShort', 'ShortSaleBuy',
#        'ShortSaleCashRepayment', 'ShortSaleLimit', 'ShortSaleSell',
#        'ShortSaleTodayBalance', 'ShortSaleYesterdayBalance', 'Note',
#        'BuyAfterSale', 'Trading_Volume', 'BuyAmount', 'SellAmount'],
#       dtype='object')

,Unnamed: 0,stock_id,stock_name,ISIN_Code,Listing_date,Listing_category,Industry_category,CFICode,date,Volume,...,ShortSaleCashRepayment,ShortSaleLimit,ShortSaleSell,ShortSaleTodayBalance,ShortSaleYesterdayBalance,Note,BuyAfterSale,Trading_Volume,BuyAmount,SellAmount
0,0,1101,台泥,TW0001101004,1962-02-09,上市,水泥工業,ESVUFR,2010-01-04,8299290.0,...,0.0,823043.0,14.0,756.0,769.0,,NaN,NaN,NaN,NaN
1,1,1101,台泥,TW0001101004,1962-02-09,上市,水泥工業,ESVUFR,2010-01-05,48059367.0,...,1.0,823043.0,172.0,840.0,756.0,,NaN,NaN,NaN,NaN
2,2,1101,台泥,TW0001101004,1962-02-09,上市,水泥工業,ESVUFR,2010-01-06,36026739.0,...,0.0,823043.0,227.0,1025.0,840.0,,NaN,NaN,NaN,NaN
3,3,1101,台泥,TW0001101004,1962-02-09,上市,水泥工業,ESVUFR,2010-01-07,14531020.0,...,0.0,823043.0,97.0,968.0,1025.0,,NaN,NaN,NaN,NaN
4,4,1101,台泥,TW0001101004,1962-02-09,上市,水泥工業,ESVUFR,2010-01-08,10671139.0,...,0.0,823043.0,330.0,1153.0,968.0,,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4078032,2775,9962,有益,TW0009962001,2006-07-10,上櫃,鋼鐵工業,ESVUFR,2021-04-19,3943000.0,...,0.0,22555.0,0.0,0.0,0.0,X,＊,1429000.0,18091350.0,17987000.0
4078033,2776,9962,有益,TW0009962001,2006-07-10,上櫃,鋼鐵工業,ESVUFR,2021-04-20,2068000.0,...,0.0,22555.0,0.0,0.0,0.0,X,＊,246000.0,3180600.0,3116500.0
4078034,2777,9962,有益,TW0009962001,2006-07-10,上櫃,鋼鐵工業,ESVUFR,2021-04-21,1226000.0,...,0.0,22555.0,0.0,0.0,0.0,NaN,＊,247000.0,3062300.0,3087000.0
4078035,2778,9962,有益,TW0009962001,2006-07-10,上櫃,鋼鐵工業,ESVUFR,2021-04-22,4908000.0,...,0.0,22555.0,0.0,0.0,0.0,NaN,＊,1898000.0,25505050.0,24705750.0
